In [16]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import csv
%load_ext autoreload
%autoreload 2
from IPython import display
from proj1_helpers import *
import costs
from split_data import *
from least_squares import *
import standardize
from gradient_descent import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Evaluate a model

def prediction(y, tX, w_star):
    """Evaluates a model in a friendly manner"""
    pred = np.dot(tX, w_star)

    pred[pred > 0] = 1
    pred[pred <= 0] = -1

    right = np.sum(pred == y)
    wrong = len(pred) - right

    print("Good prediction: %i/%i (%.3f%%)\nWrong prediction: %i/%i (%.3f%%)" %
          (right, len(y), 100.0 * float(right) / float(len(y)),
           wrong, len(y), 100.0 * float(wrong) / float(len(y))))

In [18]:
# Load the whole data

y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')

# print(headers)


In [8]:
# Correlation analysis
from scipy.stats.stats import pearsonr
from numpy import corrcoef

print(x_train.shape)
correlation_matrix = np.zeros((x_train.shape[1],x_train.shape[1]))
for i in range(x_train.shape[1]):
    for j in range(x_train.shape[1]):
        #print(headers[i+2], headers[j+2])
        corr = corrcoef(x_train[:,i], x_train[:,j])
        #print(corr[1,0])
        correlation_matrix[i,j] = corr[1,0]


#print(correlation_matrix)
        
        
        
    

(250000, 30)


In [5]:
from build_polynomial import build_poly
from least_squares import least_squares
from ridge_regression import ridge_regression

# REALY MAKES THE PREDICTION

#y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
#y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')


ratio = 0.8

for deg in range(10,11):
    for i in range (3):
        print(i)
        jetMaskTrain = x_train[:,22]==i
        jetMaskTest = x_test[:,22]==i
        if (i == 2):
            jetMaskTrain = np.asarray(x_train[:,22]==i) + np.asarray(x_train[:,22]==3) 
            jetMaskTest = np.asarray(x_test[:,22]==i) + np.asarray(x_test[:,22]==3) 
        print(len(x_train[jetMaskTrain]))
        trainProcessed, testProcessed = processing(x_train[jetMaskTrain], x_test[jetMaskTest], jet_mod = True, jet_num=i)
        #show_x(trainProcessed)
        #show_x(testProcessed)

        #jetDividedTrainProcessed.append(trainProcessed)
        #jetDividedTestProcessed.append(testProcessed)


        yJetTrain = y_train[jetMaskTrain]
        yJetTest = y_test[jetMaskTest]
        #x_train2, y_train2, x_test2, y_test2 = split_data(trainProcessed, yJetTrain, ratio)
        x_train2, y_train2, x_test2, y_test2 = trainProcessed, yJetTrain,testProcessed,yJetTest
        trainProcessedPoly = build_poly(x_train2, deg)
        testProcessedPoly = build_poly(x_test2,deg)
        loss, w_star = ridge_regression(yJetTrain, trainProcessedPoly,0.00001)
        print("Loss = %f"%(loss))
        y_pred = predict_labels(w_star, testProcessedPoly)
        

        create_csv_submission(ids_test[jetMaskTest], y_pred, "LeastSquaresPoly"+str(i))
        #loss, w_star = least_squares(y_train2, trainProcessedPoly)
        #print("Loss = %f"%(loss))
        #prediction(y_test2, testProcessedPoly, w_star)

0
99913
Nombre de variables restantes : 18
Loss = 10.295381
1
77544
Nombre de variables restantes : 22
Loss = 10.735226
2
72543
Nombre de variables restantes : 29
Loss = 0.265805


In [55]:
#y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
#y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')

#print(headers)


def processing(x_train, x_test, long_tails = [0,1,2,3,5,8,9,10,13,16,19,21,23,26,29], jet_mod = False, jet_num=0):
    
    
    # Log transform all the long tails
    correct_mask_train = x_train!=-999
    correct_mask_test = x_test!=-999
    for i in range(len(x_train[0])):    
        array_tr = x_train[correct_mask_train[:,i],i] # Only the correct Values
        array_te = x_test[correct_mask_test[:,i],i] # Only the correct Values
        if i in long_tails: # We log them if they have long tails
            array_tr = np.log(array_tr+1)
            array_te = np.log(array_te+1)
            
            
    if jet_mod:
        if jet_num == 0:
            jetFeaturesExcluded = [4,5,6,12,22,23,24,25,26,27,28,29]
        elif jet_num == 1:
            jetFeaturesExcluded = [4,5,6,12,22,26,27,28]
        else :
            jetFeaturesExcluded = [22]
    else:
        jetFeaturesExcluded = []
        
    all_indices = np.arange(x_train.shape[1])
    excepted = np.setdiff1d(all_indices,jetFeaturesExcluded)
    x_test = x_test[:,excepted]
    x_train = x_train[:,excepted]
    print("Nombre de variables restantes : " + str(len(x_train[0])))
    #print(x_train)

    
    #x_train_extended = x_train.copy()
    missing_mask_train = x_train==-999
    correct_mask_train = x_train!=-999
    
    #x_test_extended = x_test.copy()
    missing_mask_test = x_test==-999
    correct_mask_test = x_test!=-999


    for i in range(len(x_train[0])):

        array_tr = x_train[correct_mask_train[:,i],i] # Only the correct Values
        array_te = x_test[correct_mask_test[:,i],i] # Only the correct Values


        arraytr, mean_train, std_train = (standardize.standardize(array_tr))
        arrayte, mean_train, std_train = (standardize.standardize(array_te, mean_train, std_train))
        x_train[correct_mask_train[:,i],i] = arraytr
        x_test[correct_mask_test[:,i],i] = arrayte
        #plt.hist(arraytr, 10)
        #plt.title("Variable %i"%(i))
        #plt.show()


    x_train[missing_mask_train] = 0
    x_test[missing_mask_test] = 0
    
    # Now everything is normalized, let us compute some products and differences. They will be polynomial transformed too
    angle = [15,18,20]
    
    if jet_mod and (jet_num == 0 or jet_num == 1):
        angle = [12,15,17]
    diff1x_train = np.abs(x_train[:,angle[0]] - x_train[:,angle[1]]).reshape((len(x_train),1))
    diff2x_train = np.abs(x_train[:,angle[0]] - x_train[:,angle[2]]).reshape((len(x_train),1))
    diff3x_train = np.abs(x_train[:,angle[2]] - x_train[:,angle[1]]).reshape((len(x_train),1))
    
    diff1x_test = np.abs(x_test[:,angle[0]] - x_test[:,angle[1]]).reshape((len(x_test),1))
    diff2x_test = np.abs(x_test[:,angle[0]] - x_test[:,angle[2]]).reshape((len(x_test),1))
    diff3x_test = np.abs(x_test[:,angle[2]] - x_test[:,angle[1]]).reshape((len(x_test),1))
    
    #print(diff1x_train.shape)
    #print(diff2x_train.shape)
    #print(diff3x_train.shape)
    #print(x_train.shape)
    x_train = np.hstack((x_train, diff1x_train,diff2x_train,diff3x_train))
    x_test = np.hstack((x_test, diff1x_test,diff2x_test,diff3x_test))
    
    taille = len(x_train[0])
    for i in range(taille):
        for j in range(i):
            new_column_train =(x_train[:,i]*x_train[:,j]).reshape((len(x_train),1))
            x_train = np.hstack((x_train,new_column_train))
            new_column_test = (x_test[:,i]*x_test[:,j]).reshape((len(x_test),1))
            x_test = np.hstack((x_test,new_column_test))
    
    return x_train,x_test

def show_x(x):
    for i in range(len(x[0])):
        array = x[:,i]
        plt.hist(array, 250)
        plt.title("Variable %i: %s"%(i, headers[i+2]))
        plt.show()
    return x
    

#x_train, x_test = processing(x_train, x_test)
#show_x(x_test)

#print(x_train.shape)

In [57]:
from build_polynomial import build_poly
from least_squares import least_squares
from ridge_regression import ridge_regression

# EVALUATE A MODEL

#y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
#y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')


ratio = 0.8

for deg in range(2,18,2):
    print("Degré : " + str(deg))
    for i in range (3):
        print("Jet : " + str(i))
        jetMaskTrain = x_train[:,22]==i
        jetMaskTest = x_test[:,22]==i
        if (i == 2):
            jetMaskTrain = np.asarray(x_train[:,22]==i) + np.asarray(x_train[:,22]==3) 
            jetMaskTest = np.asarray(x_test[:,22]==i) + np.asarray(x_test[:,22]==3) 
        #print(len(x_train[jetMaskTrain]))
        trainProcessed, testProcessed = processing(x_train[jetMaskTrain], x_test[jetMaskTest], jet_mod = True, jet_num=i)
        #show_x(trainProcessed)
        #show_x(testProcessed)

        #jetDividedTrainProcessed.append(trainProcessed)
        #jetDividedTestProcessed.append(testProcessed)


        yJetTrain = y_train[jetMaskTrain]
        yJetTest = y_test[jetMaskTest]
        x_train2, y_train2, x_test2, y_test2 = split_data(trainProcessed, yJetTrain, ratio)
        #x_train2, y_train2, x_test2, y_test2 = trainProcessed, yJetTrain,testProcessed,yJetTest
        trainProcessedPoly = build_poly(x_train2, deg)
        testProcessedPoly = build_poly(x_test2,deg)
        loss, w_star = least_squares(y_train2, trainProcessedPoly)
        #print("Loss = %f"%(loss))
        y_pred = predict_labels(w_star, testProcessedPoly)
        

        #create_csv_submission(ids_test[jetMaskTest], y_pred, "LeastSquaresPoly"+str(i))
        prediction(y_test2, testProcessedPoly, w_star)

Degré : 2
Jet : 0
Nombre de variables restantes : 18
Good prediction: 16760/19983 (83.871%)
Wrong prediction: 3223/19983 (16.129%)
Jet : 1
Nombre de variables restantes : 22
Good prediction: 12144/15509 (78.303%)
Wrong prediction: 3365/15509 (21.697%)
Jet : 2
Nombre de variables restantes : 29
Good prediction: 11760/14509 (81.053%)
Wrong prediction: 2749/14509 (18.947%)
Degré : 4
Jet : 0
Nombre de variables restantes : 18
Good prediction: 16833/19983 (84.237%)
Wrong prediction: 3150/19983 (15.763%)
Jet : 1
Nombre de variables restantes : 22
Good prediction: 12129/15509 (78.206%)
Wrong prediction: 3380/15509 (21.794%)
Jet : 2
Nombre de variables restantes : 29
Good prediction: 11978/14509 (82.556%)
Wrong prediction: 2531/14509 (17.444%)
Degré : 6
Jet : 0
Nombre de variables restantes : 18
Good prediction: 16809/19983 (84.116%)
Wrong prediction: 3174/19983 (15.884%)
Jet : 1
Nombre de variables restantes : 22
Good prediction: 12456/15509 (80.315%)
Wrong prediction: 3053/15509 (19.685%)
Je

KeyboardInterrupt: 

In [ ]:
from ridge_regression import ridge_regression
from build_polynomial import build_poly
from plots import cross_validation_visualization
from build_k_indices import build_k_indices
from cross_validation import cross_validation
import numpy as np

def cross_validation_demo():
    seed = 26
    k_fold = 2
    
    lambdas = np.logspace(-5, -2, 2)
    degrees = np.linspace(6, 15,2)
    
    y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
    y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')
    x_train, x_test = processing(x_train, x_test)
    
    losses = np.zeros((len(lambdas), len(degrees)))



    y = y_train
    x = x_train
    
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    #print(k_indices)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    # cross validation: TODO
    # ***************************************************
    for ind, lambda_ in enumerate(lambdas):
        print(ind)

        for ind_deg, degree in enumerate(degrees):
            degree = int(degree)
            print(degree)
            mse_tr_ = []
            mse_te_ = []
            x = build_poly(x_train, degree)
            y = y_train
            for k in range(k_fold):
                print(k)
                #print(k_indices[k])
                mse_tr_i, mse_te_i = cross_validation(y, x, k_indices, k, lambda_, degree)
                mse_tr_.append(mse_tr_i)
                mse_te_.append(mse_te_i)
            losses[ind,ind_deg] = np.sqrt(2*np.asarray(mse_te_).mean())
            #rmse_tr.append(np.sqrt(2*np.asarray(mse_tr_).mean()))
            #rmse_te.append(np.sqrt(2*np.asarray(mse_te_).mean()))
    #print(rmse_tr)
    #print(rmse_te)
    #cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    min_row, min_col = np.unravel_index(np.argmin(losses), losses.shape)
    print(losses[min_row, min_col], lambdas[min_row], degrees[min_col])

cross_validation_demo()



0
6
0
1
15
0


In [11]:
from build_polynomial import build_poly
from ridge_regression import ridge_regression


ratio = 0.8
lambda_ = 0.001
y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')
x_train, x_test = processing(x_train, x_test)
x_train2, y_train2, x_test2, y_test2 = split_data(x_train, y_train, ratio)
x_train2 = build_poly(x_train2,14)
x_test2 = build_poly(x_test2,14)
print(x_train2.shape)

loss, w_star = ridge_regression(y_train2, x_train2, lambda_)
print("Loss = %f"%(loss))
prediction(y_test2, x_test2, w_star)

(200000, 420)
Loss = 0.266692
Good prediction: 41103/50000 (82.206%)
Wrong prediction: 8897/50000 (17.794%)


In [7]:
from build_polynomial import build_poly

y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')
x_train, x_test = processing(x_train, x_test)
x_train2, y_train2, x_test2, y_test2 = x_train, y_train, x_test, y_test  #split_data(x_train, y_train, ratio)
print(x_train2.shape)

x_train2 = build_poly(x_train2,17)
x_test2 = build_poly(x_test2,17)

loss, w_star = least_squares(y_train2, x_train2)
print("Loss = %f"%(loss))
y_pred = predict_labels(w_star, x_test2)

create_csv_submission(ids_test, y_pred, "LeastSquaresPoly7")

KeyboardInterrupt: 

In [ ]:
# from gradient_descent import *
from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 2
gamma = 0.1

# Initialization
w_initial = np.ones((x_train.shape[1],1))

# Start gradient descent.

gradient_losses, gradient_ws = gradient_descent(y_train2, x_train2, w_initial, max_iters, gamma)


# Print result

prediction(y_test2, x_test2, gradient_ws)

In [22]:
rmse_tr = [321302705.32118189, 3081770291.9105225, 476655556.38870007, 6586530333.0643339, 3926785041.4215159, 1616028753.2465477, 3052424875.4092693, 754319853.14006972, 820583378.54368246, 17035954628.916489, 5638215798.1593828, 552445902.43237698, 1247083919.8717456, 523056402.48754984, 815035142.10989654, 1237961483.8206384, 472263250.85445482, 1128685585.2602913, 337359441.7431621, 444615308.73539412, 1187695089.0441186, 1154291756.4891524, 633935548.60643542, 893150866.4781518, 599195433.74783516, 501288433.01773316, 747877161.89168787, 949075067.35166752, 568752240.36924183, 500503541.8066045]
rmse_te = [321459366.76115286, 3080249672.8730979, 475708961.98398209, 6580836712.3314152, 3924229065.938735, 1613714218.0849831, 3047141985.7805061, 752419169.80420673, 820506657.64361334, 17050315097.83621, 5652826434.3759661, 549735520.04158115, 1249299198.0115349, 523593309.05457181, 816846389.92515397, 1233223624.2504148, 472515141.34812337, 1125814263.3493133, 337950445.59918481, 442503614.14081073, 1187863323.660614, 1153379569.6909783, 633791488.363711, 892879869.87547123, 599439412.66542077, 501843482.74288714, 748023611.58240223, 950631904.87251222, 566716174.01105654, 501018160.37078071]

print(np.argmin(rmse_te))
i = 0
for j in (np.logspace(-4, 0, 30)):
    i +=1
    if i==np.argmin(rmse_te):
        print(j)

0


In [14]:
print(np.asarray([3,2]) * np.asarray([3,2]))

array([9, 4])

In [22]:
np.asarray([True, False, True]) + np.asarray([False, False, True])

array([ True, False,  True], dtype=bool)

In [19]:
y_train, x_train, ids_train, headers = load_csv_data('data/train.csv')
y_test, x_test, ids_test, headers_test = load_csv_data('data/test.csv')

print(headers)

x_train_extended = x_train.copy()
x_train_extended = x_train_extended==-999


print(x_train_extended.shape)
print(x_train.shape)
nbr_param = len(x_train[0])
print("Number of variables: %i"%(nbr_param))

# Plot the histograms


for i in range(len(x_train[0])):
    array_ = x_train[:,i]
    array = (standardize.standardize(array_[array_ != -999]))
    x_train[array_!=-999,i] = array
    n, bins, patches = plt.hist(array[array != -999], 250)
    #n, bins, patches = plt.hist(array, 50)
    plt.title("Variable %i: %s"%(i, headers[i+2]))
    plt.show()

x_train_extended = np.concatenate((np.uint(x_train_extended),x_train),axis=1)
print(x_train_extended.shape)
print(x_train.shape)

['Id' 'Prediction' 'DER_mass_MMC' 'DER_mass_transverse_met_lep'
 'DER_mass_vis' 'DER_pt_h' 'DER_deltaeta_jet_jet' 'DER_mass_jet_jet'
 'DER_prodeta_jet_jet' 'DER_deltar_tau_lep' 'DER_pt_tot' 'DER_sum_pt'
 'DER_pt_ratio_lep_tau' 'DER_met_phi_centrality' 'DER_lep_eta_centrality'
 'PRI_tau_pt' 'PRI_tau_eta' 'PRI_tau_phi' 'PRI_lep_pt' 'PRI_lep_eta'
 'PRI_lep_phi' 'PRI_met' 'PRI_met_phi' 'PRI_met_sumet' 'PRI_jet_num'
 'PRI_jet_leading_pt' 'PRI_jet_leading_eta' 'PRI_jet_leading_phi'
 'PRI_jet_subleading_pt' 'PRI_jet_subleading_eta' 'PRI_jet_subleading_phi'
 'PRI_jet_all_pt']
(250000, 30)
(250000, 30)
Number of variables: 30


ValueError: setting an array element with a sequence.